# 03 Merge With Hash Info

In [1]:
from libraries import *
from parameters import *

In [2]:
%load_ext rpy2.ipython

In [3]:
os.getcwd()
os.chdir(projectDir)

***

In [4]:
adata = sc.read('outputs/anndata/adata.h5ad')

FileNotFoundError: [Errno 2] Unable to open file (unable to open file: name = 'outputs/anndata/adata.h5ad', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [12]:
adata.obs.shape

(1071671, 13)

In [13]:
from pathlib import Path
from tqdm.auto import tqdm

In [14]:
hash_ads = {}
feats = {}

for d in tqdm(sorted([x for x in Path('data').iterdir() if x.is_dir() and x.name.startswith('E')])):
    sample = d.name.replace('_', '-')
    roundlow = sample.split('-')[0].lower()
    chrom = sample.split('-')[1].lower()
    
    hash_path = Path('data') / sample.replace('-', '_') / f'{roundlow}_ch{chrom}_mixhash' / f'{roundlow}_ch{chrom}_mixhash_demux.h5ad'
    if hash_path.is_file():
        print(sample)
        hash_ads[sample] = sc.read(hash_path)
        
    feat_path = Path(f'data/{sample.replace("-", "_")}/R{roundlow[1:]}_Ch{chrom}_FeatBarc/R{roundlow[1:]}_Ch{chrom}_FeatBarc.csv')
    if feat_path.is_file():
        print(sample)
        feats[sample] = pd.read_csv(feat_path)

  0%|          | 0/48 [00:00<?, ?it/s]

E3-1
E3-1
E3-10
E3-10
E3-11
E3-11
E3-12
E3-12
E3-13
E3-13
E3-14
E3-14
E3-15
E3-15
E3-16
E3-16
E3-17
E3-17
E3-18
E3-18
E3-19
E3-19
E3-2
E3-2
E3-20
E3-20
E3-21
E3-21
E3-22
E3-22
E3-23
E3-23
E3-24
E3-24
E3-3
E3-3
E3-4
E3-4
E3-5
E3-5
E3-6
E3-6
E3-7
E3-7
E3-8
E3-8
E3-9
E3-9
E4-1
E4-1
E4-10
E4-10
E4-11
E4-11
E4-12
E4-12
E4-13
E4-13
E4-14
E4-14
E4-15
E4-15
E4-16
E4-16
E4-17
E4-17
E4-19
E4-19
E4-2
E4-2
E4-20
E4-20
E4-21
E4-21
E4-22
E4-22
E4-24
E4-24
E4-3
E4-3
E4-4
E4-4
E4-5
E4-5
E4-6
E4-6
E4-7
E4-7
E4-8
E4-8
E4-9
E4-9


## Merge hash IDs

In [15]:
hash_ads['E3-1'].obs.head()

,Channel,n_genes,n_counts,demux_type,assignment
index,,,,,
AAACCCAAGCGTTCCG,,1878,5915.0,singlet,100LPS_B
AAACCCAAGCTTCATG,,2425,7652.0,doublet,"100LPS_G,100LPS_C"
AAACCCAAGGAGATAG,,1412,3452.0,singlet,100LPS_B
AAACCCAAGTACAGCG,,185,245.0,singlet,100LPS_H
AAACCCAAGTCCGCGT,,1715,4448.0,singlet,100LPS_G


In [16]:
%%time

hash_ad = sc.AnnData.concatenate(*hash_ads.values(), 
                                 batch_key='mixhash', 
                                 batch_categories=list(hash_ads.keys()))
hash_ad

CPU times: user 9.97 s, sys: 7.63 s, total: 17.6 s
Wall time: 17.6 s


AnnData object with n_obs × n_vars = 1258502 × 31053
    obs: 'Channel', 'n_genes', 'n_counts', 'demux_type', 'assignment', 'mixhash'
    var: 'gene_ids', 'robust'
    obsm: 'raw_probs'

In [17]:
hash_ad.obs.head()

,Channel,n_genes,n_counts,demux_type,assignment,mixhash
index,,,,,,
AAACCCAAGCGTTCCG-E3-1,,1878,5915.0,singlet,100LPS_B,E3-1
AAACCCAAGCTTCATG-E3-1,,2425,7652.0,doublet,"100LPS_G,100LPS_C",E3-1
AAACCCAAGGAGATAG-E3-1,,1412,3452.0,singlet,100LPS_B,E3-1
AAACCCAAGTACAGCG-E3-1,,185,245.0,singlet,100LPS_H,E3-1
AAACCCAAGTCCGCGT-E3-1,,1715,4448.0,singlet,100LPS_G,E3-1


In [18]:
hash_ad.obs.index = ['-'.join([x.split('-')[0], 
                               '1', 
                               x.split('-')[1], 
                               x.split('-')[2]]) for x in hash_ad.obs.index]

In [19]:
hash_ad.obs.head()

,Channel,n_genes,n_counts,demux_type,assignment,mixhash
AAACCCAAGCGTTCCG-1-E3-1,,1878,5915.0,singlet,100LPS_B,E3-1
AAACCCAAGCTTCATG-1-E3-1,,2425,7652.0,doublet,"100LPS_G,100LPS_C",E3-1
AAACCCAAGGAGATAG-1-E3-1,,1412,3452.0,singlet,100LPS_B,E3-1
AAACCCAAGTACAGCG-1-E3-1,,185,245.0,singlet,100LPS_H,E3-1
AAACCCAAGTCCGCGT-1-E3-1,,1715,4448.0,singlet,100LPS_G,E3-1


In [20]:
adata.obs.head()

,barcode_rank,channel,empty_drops_FDR,log10_n_genes,log10_n_umis,mt_frac,n_counts,n_genes,n_umis,neg_log10_empty_drops_FDR,raw,round,sample_name
AAACCCAAGCGTTCCG-1-E3-1,4441.0,1,0.0,3.273696,3.771955,0.064243,5915.0,1878,5915.0,4.0,True,E3,E3-1
AAACCCAAGCTTCATG-1-E3-1,1588.0,1,0.0,3.384712,3.883775,0.062729,7652.0,2425,7652.0,4.0,True,E3,E3-1
AAACCCAAGGAGATAG-1-E3-1,15850.0,1,0.0,3.149835,3.538071,0.043163,3452.0,1412,3452.0,4.0,True,E3,E3-1
AAACCCAAGTCCGCGT-1-E3-1,9651.5,1,0.0,3.234264,3.648165,0.071493,4448.0,1715,4448.0,4.0,True,E3,E3-1
AAACCCACAAAGGCTG-1-E3-1,1517.0,1,0.0,3.382017,3.887898,0.074822,7725.0,2410,7725.0,4.0,True,E3,E3-1


In [21]:
adata.shape

(1071671, 13811)

In [22]:
hash_ad.shape

(1258502, 31053)

In [23]:
len(set(adata.obs.index.tolist()) & set(hash_ad.obs.index.tolist()))

1071671

In [24]:
adata.obs['demux_type'] = hash_ad.obs.demux_type
adata.obs['hash_assignment'] = hash_ad.obs.assignment

In [25]:
adata.obs.shape

(1071671, 15)

In [26]:
hash_ad.obs.tail()

,Channel,n_genes,n_counts,demux_type,assignment,mixhash
TTTGTTGTCATTCTTG-1-E4-9,,100,115.0,unknown,,E4-9
TTTGTTGTCCGTCACT-1-E4-9,,107,127.0,unknown,,E4-9
TTTGTTGTCCGTGGTG-1-E4-9,,1771,4500.0,singlet,100LPS_B,E4-9
TTTGTTGTCGACCAAT-1-E4-9,,2608,7824.0,doublet,"100LPS_D,100LPS_E",E4-9
TTTGTTGTCTCAACCC-1-E4-9,,2109,6072.0,singlet,100LPS_C,E4-9


In [27]:
adata.obs.head()

,barcode_rank,channel,empty_drops_FDR,log10_n_genes,log10_n_umis,mt_frac,n_counts,n_genes,n_umis,neg_log10_empty_drops_FDR,raw,round,sample_name,demux_type,hash_assignment
AAACCCAAGCGTTCCG-1-E3-1,4441.0,1,0.0,3.273696,3.771955,0.064243,5915.0,1878,5915.0,4.0,True,E3,E3-1,singlet,100LPS_B
AAACCCAAGCTTCATG-1-E3-1,1588.0,1,0.0,3.384712,3.883775,0.062729,7652.0,2425,7652.0,4.0,True,E3,E3-1,doublet,"100LPS_G,100LPS_C"
AAACCCAAGGAGATAG-1-E3-1,15850.0,1,0.0,3.149835,3.538071,0.043163,3452.0,1412,3452.0,4.0,True,E3,E3-1,singlet,100LPS_B
AAACCCAAGTCCGCGT-1-E3-1,9651.5,1,0.0,3.234264,3.648165,0.071493,4448.0,1715,4448.0,4.0,True,E3,E3-1,singlet,100LPS_G
AAACCCACAAAGGCTG-1-E3-1,1517.0,1,0.0,3.382017,3.887898,0.074822,7725.0,2410,7725.0,4.0,True,E3,E3-1,singlet,100LPS_H


In [28]:
adata.obs.head()

,barcode_rank,channel,empty_drops_FDR,log10_n_genes,log10_n_umis,mt_frac,n_counts,n_genes,n_umis,neg_log10_empty_drops_FDR,raw,round,sample_name,demux_type,hash_assignment
AAACCCAAGCGTTCCG-1-E3-1,4441.0,1,0.0,3.273696,3.771955,0.064243,5915.0,1878,5915.0,4.0,True,E3,E3-1,singlet,100LPS_B
AAACCCAAGCTTCATG-1-E3-1,1588.0,1,0.0,3.384712,3.883775,0.062729,7652.0,2425,7652.0,4.0,True,E3,E3-1,doublet,"100LPS_G,100LPS_C"
AAACCCAAGGAGATAG-1-E3-1,15850.0,1,0.0,3.149835,3.538071,0.043163,3452.0,1412,3452.0,4.0,True,E3,E3-1,singlet,100LPS_B
AAACCCAAGTCCGCGT-1-E3-1,9651.5,1,0.0,3.234264,3.648165,0.071493,4448.0,1715,4448.0,4.0,True,E3,E3-1,singlet,100LPS_G
AAACCCACAAAGGCTG-1-E3-1,1517.0,1,0.0,3.382017,3.887898,0.074822,7725.0,2410,7725.0,4.0,True,E3,E3-1,singlet,100LPS_H


In [29]:
adata.obs['hash_assignment'] = adata.obs['hash_assignment'].astype('category')

In [30]:
adata.obs['hash_assignment'].cat.categories

Index(['', '100LPS_A', '100LPS_A,100LPS_B', '100LPS_A,100LPS_B,100LPS_C',
       '100LPS_A,100LPS_B,100LPS_C,100LPS_D',
       '100LPS_A,100LPS_B,100LPS_C,100LPS_E',
       '100LPS_A,100LPS_B,100LPS_C,100LPS_F',
       '100LPS_A,100LPS_B,100LPS_C,100LPS_F,100LPS_D',
       '100LPS_A,100LPS_B,100LPS_C,100LPS_F,100LPS_E',
       '100LPS_A,100LPS_B,100LPS_C,100LPS_F,100LPS_H',
       ...
       '100LPS_H,100LPS_G,100LPS_D,100LPS_E,100LPS_A',
       '100LPS_H,100LPS_G,100LPS_D,100LPS_F',
       '100LPS_H,100LPS_G,100LPS_D,100LPS_F,100LPS_B',
       '100LPS_H,100LPS_G,100LPS_E', '100LPS_H,100LPS_G,100LPS_E,100LPS_A',
       '100LPS_H,100LPS_G,100LPS_E,100LPS_C',
       '100LPS_H,100LPS_G,100LPS_E,100LPS_D',
       '100LPS_H,100LPS_G,100LPS_E,100LPS_F', '100LPS_H,100LPS_G,100LPS_F',
       '100LPS_H,100LPS_G,100LPS_F,100LPS_D'],
      dtype='object', length=2142)

Filter out doublets and multiplets


In [31]:
adata = adata[adata.obs.demux_type == 'singlet']

/home/beraslan/anaconda3/lib/python3.7/site-packages/pandas/core/arrays/categorical.py:2487: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  res = method(*args, **kwargs)


In [32]:
adata.obs.shape

(838201, 15)

In [33]:
adata.write('outputs/anndata/adataSinglets.h5ad')